In [3]:
"""
MEMBERS
Priyanshu Arya (21324019)
Garima Mandli (21113048)​
Anvita Deepak Prasad (20122010)​
Roopa Pramodan (21324021)​
Shoilayee Chaudhuri (21112102)
"""

'\nMEMBERS\nPriyanshu Arya (21324019)\nGarima Mandli (21113048)\u200b\nAnvita Deepak Prasad (20122010)\u200b\nRoopa Pramodan (21324021)\u200b\nShoilayee Chaudhuri (21112102)\n'

In [4]:
# !python -m spacy download en_core_web_sm

In [5]:
from scrapy import *
from bs4 import BeautifulSoup
import urllib.request

import pandas as pd
import numpy as np

import os

from bs4 import BeautifulSoup
import urllib.request
import os
import requests

import spacy
nlp =spacy.load("en_core_web_sm")

from spacy.lang.en.stop_words import STOP_WORDS

df=pd.read_excel("Input/Input.xlsx")

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment" #using a pretrained model 

tokenizer = tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

import re

/home/yash/anaconda3/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [6]:

ps = PorterStemmer()


#This is used to Calculate the Average postive negative and neutrality score
def ScoreCalculator(FileName):

    f=open(FileName, 'r')
    NegativeScore=0    
    PositiveScore=0
    NeutralScore=0
    number=0
    StringWord=[]
    ExceptionEncountered=False
    Scores_exp=[0,0,0]

    for line in f:
        number+=1
        try:
            encoded_text = tokenizer(line, return_tensors='pt')
            output = model(**encoded_text)
            scores = output[0][0].detach().numpy()
            scores = softmax(scores)
            NegativeScore=NegativeScore+scores[0]
            NeutralScore=NeutralScore+scores[1]
            PositiveScore=PositiveScore+scores[2]
        except:
            StringWord.append(line)
            ExceptionEncountered=True
            

    if(ExceptionEncountered):
        Scores_exp=ExceptionHandler(StringWord)       
        Average_dict = {
            'roberta_NegativeScore' : (NegativeScore+Scores_exp["NegativeScore"])/(number+1),
            'roberta_NeutralScore' : (NeutralScore+Scores_exp["NeutralScore"])/(number+1),
            'roberta_PositiveScore' : (PositiveScore+Scores_exp["PositiveScore"])/(number+1)
        }
    else:
         Average_dict = {
            'roberta_NegativeScore' : NegativeScore/number,
            'roberta_NeutralScore' : NeutralScore/number,
            'roberta_PositiveScore' : NeutralScore/number,
        }       
    
    return(NegativeScore/number,NeutralScore/number,NeutralScore/number,)

#This function is used to handle the Exception thrown by the Model as the model does not support tensor larger than 520
def ExceptionHandler(String):
    NegativeScore=0    
    PositiveScore=0
    NeutralScore=0
    StringNeed=[]
    StringProcess=""
    itr=0
    length=len(String)
    TotalNeu=0
    TotalNeg=0
    TotalPost=0
    for StringItem in String:
        number =0
        StringNeed=StringItem.split()
        for item in StringNeed:
            if(number%25==0 and number !=0):
                scores=ModelOuput(StringProcess)
                StringProcess=""
                NegativeScore=NegativeScore+scores[0]
                NeutralScore=NeutralScore+scores[1]
                PositiveScore=PositiveScore+scores[2]
            else:
                StringProcess=StringProcess+" "+item
            number+=1
        TotalPost=(PositiveScore/number)+TotalPost
        TotalNeg=(NegativeScore/number)+TotalNeg
        TotalNeu=(NeutralScore/number)+TotalNeu


    Average_local = {
        'NegativeScore' : TotalNeg/length,
        'NeutralScore' : TotalNeu/length,
        'PositiveScore' : TotalPost/length
    }


    return(Average_local)


def ModelOuput(String):

    encoded_text = tokenizer(String, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    return(scores)


def PolarityCal(TotalWords,Neg,Positive):

    PolarityScore=((Positive-Neg)/(Positive+Neg))+0.000001

    Subjectivity=((Positive+Neg)/TotalWords)+ 0.000001

    return(PolarityScore,Subjectivity)

def CountWords(filename):
    NoComplex=0
    NoSyllabus=0
    WordCount=0
    with open(filename, 'r') as file:
        content = file.read()
        words = content.split()
        TotalWords = len(words)
        for itere in range(TotalWords):
            if(ComplexWord(words[itere])):
                NoComplex+=1  
            NoSyllabus=NoSyllabus+syllabus(ps.stem(words[itere]))
             
    return (TotalWords,NoComplex,NoSyllabus)

def ComplexWord(words):
    count=0
    for char in range(len(words)):
            if (words[char] == 'a' or words[char] == 'e' or words[char] == 'i' or words[char] == 'o' or words[char] == 'u' or words[char] == 'A' or words[char] == 'E' or words[char] == 'I' or words[char] == 'O' or words[char] == 'U'):
                count+=1
    if(count>=2):
        return(True)
    else:
        return(False)   
    
def Sentence(FileName):
    num_lines=0
    number=0
    with open(FileName,'r') as file:
        for line in file:
            number+=1
            num_lines += line.count(".")
            num_lines += line.count("?")
            num_lines += line.count(":-")
    return(num_lines)

def syllabus(word):
    count=0
    for char in range(len(word)):
            if (word[char] == 'a' or word[char] == 'e' or word[char] == 'i' or word[char] == 'o' or word[char] == 'u' or word[char] == 'A' or word[char] == 'E' or word[char] == 'I' or word[char] == 'O' or word[char] == 'U'):
                count+=1
    return(count)

def pronouns(FileName):
    pronouns=0
    pro = re.compile(r'\b(I|he|she|we|my|ours|(?-i:us))\b',re.I)
    with open(FileName,'r') as file:
        for line in file:
            pronouns = pronouns+len(pro.findall(line))
    return(pronouns)

def WordLen(FileName):
    count=0
    with open(FileName,'r') as file:
        content = file.read()
        word = content.split()
        TotalWords = len(word)
        for itere in range(TotalWords):
            WordChar=word[itere]
            for char in range(len(WordChar)):
                count+=1
        count=count/TotalWords
    return(count)



In [7]:
def RemovalText(RemovalFile):
    #open the text file  
    with open('removal.txt', 'r') as f: 
        #read the text file into a list of lines 
        lines = f.readlines() 
        
    #create an empty dictionary 
    file_dict = []
        
        #loop through the lines in the text file  
    for line in lines: 
        line=line.replace("\n","")
        file_dict.append(line)

    FileDict=file_dict

    return(file_dict)


file_dict= RemovalText('Input/removal.txt')

In [8]:

def DataExtration(URL_ID,UrlAddress,file_dict):

    #Setting the max number of seconds to allow for processing
    timeout = 10

    #This function is used to extract the data from the articles

    URL_str=os.path.join("ArticleData/",str(URL_ID))
    URL_str=str(URL_str)+".txt"
    try:
        # urllib.request.urlretrieve(UrlAddress,"HtmlTemp.txt")
        response = requests.get(UrlAddress,timeout=timeout).text
    except:
        print("Error 404 "+UrlAddress)
        

    # file = open("HtmlTemp.txt", "r")
    # contents = file.read()
    
    soup = BeautifulSoup(response, 'html.parser')

    title = soup.title.string

    f = open(URL_str, "w")

    f.writelines(title+"\n")

    for data in soup.find_all("p"):
        sum = data.get_text()

        #Removal of unwanted data
        for element in file_dict:
            if sum == element:
                IsTrue=True
                break
            else:
                IsTrue=False

        if IsTrue:
            continue
        else:
            f.writelines(sum+"\n")

    for data in soup.find_all("ol"):
        sum = data.get_text()

        #Removal of unwanted data
        for element in file_dict:
            if sum == element:
                IsTrue=True
                break
            else:
                IsTrue=False

        if IsTrue:
            continue
        else:
            f.writelines(sum+"\n")
 
    f.close()

    return(URL_str)


def StopWordRemoval(NameFile,Url):

    String=""
    Url=os.path.join("ArticleData/ProcessedData/",str(Url)+".txt")
    f=open(Url,"w")
    file = open(NameFile,"r")

    for line in file:
        String=""
        doc=nlp(line)
        for token in doc:
            if not token.is_stop and not token.is_punct:
                String=String+" "+str(token)
        f.writelines(String+"\n")


    return(Url)
    

In [9]:
FileNameP=[]#This stores all the name of the processed article stored in the system
FileName=[]#This stores all the name of the article stored in the system
IntShape=0

#used clean and extract the data
while (IntShape!=200):
    

    try:
        name = DataExtration(df.iloc[IntShape,0],df.iloc[IntShape,1],file_dict)
        print(name)
        FileName.append(name)

        name=StopWordRemoval(name,df.iloc[IntShape,0])

        FileNameP.append(name)
        IntShape+=1

    except:
        print("Error 404 "+df.iloc[IntShape,1])
        IntShape+=1

#Used to Calculate the rest of the things

ArticleData/1.txt
ArticleData/2.txt
ArticleData/3.txt
ArticleData/4.txt
ArticleData/5.txt
ArticleData/6.txt
ArticleData/7.txt
ArticleData/8.txt
ArticleData/9.txt
ArticleData/10.txt
ArticleData/11.txt
Error 404 https://dl.acm.org/doi/pdf/10.1145/2843948
ArticleData/13.txt
ArticleData/14.txt
ArticleData/15.txt
ArticleData/16.txt
ArticleData/17.txt
ArticleData/18.txt
ArticleData/19.txt
ArticleData/20.txt
ArticleData/21.txt
ArticleData/22.txt
ArticleData/23.txt
ArticleData/24.txt
Error 404 https://www.investopedia.com/netflix-looks-to-accelerate-revenue-and-subscriber-growth-for-q4-8546779
Error 404 https://www.investopedia.com/netflix-looks-to-accelerate-revenue-and-subscriber-growth-for-q4-8546779
ArticleData/26.txt
Error 404 https://www.investopedia.com/netflix-s-better-than-expected-q4-revenue-got-a-boost-from-strong-subscriber-growth-8547829
ArticleData/28.txt
ArticleData/29.txt
ArticleData/30.txt
ArticleData/31.txt
ArticleData/32.txt
ArticleData/33.txt
ArticleData/34.txt
ArticleData/

In [10]:


df[["POSITIVE SCORE","NEGATIVE SCORE","POLARITY SCORE","SUBJECTIVITY SCORE","AVG SENTENCE LENGTH","PERCENTAGE OF COMPLEX WORDS","FOG INDEX","AVG NUMBER OF WORDS PER SENTENCE","COMPLEX WORD COUNT","WORD COUNT","SYLLABLE PER WORD","PERSONAL PRONOUNS","AVG WORD LENGTH"]]=0
IntShape=0
DataFrameInt=2
Polarity_scores=[]#Used to store the postive negative and neutral score
# TotalWordsArr=[]
# ComplexwordArr=[]
# NoSyllabusArr=[]
# SyllabussWordArr=[]
# PercentageComplexArr=[]
# polarArr=[]
# SubArr=[]
# pronounsArr=[]
# AverWordArr=[]
# SentenceLengthArr=[]
# AverageSentenceArr=[]
# FogIndexArr=[]
while (IntShape!=200):
        

    try:
        DataFrameInt=2

        #Used to store the postive negative and neutral score
        Neg,Neu,Pos=ScoreCalculator(FileNameP[IntShape])

        #Stores the total number of words, complex words amd syllabus 
        TotalWords,Complexword,NoSyllabus=CountWords(FileNameP[IntShape])

        CharCount=0
        #SYllabus per word
        SyllabussWord=(NoSyllabus/TotalWords)

        #Complex word percentage
        PercentageComplex=(Complexword/TotalWords) 
        #Polarity Score and Subjectivity Score
        polar,Sub=PolarityCal(TotalWords,Neg,Pos)

        #  #Number of Pronouns
        pronoun=pronouns(FileName[IntShape])

        #Average word length
        AverWord=WordLen(FileName[IntShape])
        # print(AverWord)
        # print(TotalWords)
        # print(SentenceLength)

        #Total number of Sentences
        SentenceLength=Sentence(FileName[IntShape])

        #Average lenght of the senetence
        AverageSentence=TotalWords/SentenceLength

        #Fog Index
        FogIndex=(0.4*(AverageSentence+PercentageComplex))


        df.iloc[IntShape,DataFrameInt]=Pos
        DataFrameInt+=1
        df.iloc[IntShape,DataFrameInt]=Neg
        DataFrameInt+=1
        df.iloc[IntShape,DataFrameInt]=polar
        DataFrameInt+=1
        df.iloc[IntShape,DataFrameInt]=Sub
        DataFrameInt+=1
        df.iloc[IntShape,DataFrameInt]=AverageSentence
        DataFrameInt+=1
        df.iloc[IntShape,DataFrameInt]=PercentageComplex
        DataFrameInt+=1
        df.iloc[IntShape,DataFrameInt]=FogIndex
        DataFrameInt+=1
        df.iloc[IntShape,DataFrameInt]=AverWord
        DataFrameInt+=1
        df.iloc[IntShape,DataFrameInt]=Complexword
        DataFrameInt+=1
        df.iloc[IntShape,DataFrameInt]=TotalWords
        DataFrameInt+=1
        df.iloc[IntShape,DataFrameInt]=SyllabussWord
        DataFrameInt+=1
        df.iloc[IntShape,DataFrameInt]="Unknown"
        DataFrameInt+=1
        df.iloc[IntShape,DataFrameInt]=AverWord
        DataFrameInt+=1    

        IntShape+=1
    
    except:
        print("Error 404 file not found "+df.iloc[IntShape,1])
        IntShape+=1


df.to_excel("output.xlsx")

/tmp/ipykernel_21424/1855751981.py:59: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.1392676698300859' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.iloc[IntShape,DataFrameInt]=Neg
/tmp/ipykernel_21424/1855751981.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.6502219766342334' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.iloc[IntShape,DataFrameInt]=polar
/tmp/ipykernel_21424/1855751981.py:63: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.003509008902910296' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.iloc[IntShape,DataFrameInt]=Sub
/tmp/ipykernel_21424/1855751981.py:65: FutureWarning: Setting an item of incompatible dtype i

Error 404 file not found https://www.investopedia.com/netflix-looks-to-accelerate-revenue-and-subscriber-growth-for-q4-8546779
Error 404 file not found https://www.sciencedirect.com/science/article/pii/S0167268123000793
Error 404 file not found https://thenextweb.com/news/how-netflix-shapes-mainstream-culture-explained-by-data
Error 404 file not found https://www.investopedia.com/is-amazon-prime-worth-it-5081871
Error 404 file not found https://www.wired.com/story/amazon-prime-perks/
Error 404 file not found https://www.vox.com/recode/2019/5/3/18511544/amazon-prime-oral-history-jeff-bezos-one-day-shipping
Error 404 file not found https://www.inc.com/jason-aten/amazon-may-have-just-quietly-added-its-single-most-important-feature.html
Error 404 file not found https://www.thehindubusinessline.com/news/variety/viacom18-completes-merger-of-jiocinema-and-voot-ott-platforms-majority-subscribers-transitioned/article67197460.ece
Error 404 file not found https://www.afaqs.com/news/ott-streaming/

Token indices sequence length is longer than the specified maximum sequence length for this model (1073 > 512). Running this sequence through the model will result in indexing errors


Error 404 file not found https://www.exchange4media.com/marketing-initiative-news/jiocinema-to-revolutionize-ipl-2023-viewing-with-unmatched-coverage-innovative-features-125447.html
Error 404 file not found https://apnews.com/article/disney-hulu-401cb68495d3c8d7edc1143a556e3f01
Error 404 file not found https://mngtnote.blogspot.com/2015/09/strategic-management-in-action-hulu.html
Error 404 file not found https://www.techadvisor.com/article/735708/disney-plus-vs-hulu.html
Error 404 file not found https://cordcutting.com/compare/disney-plus-vs-hulu/
Error 404 file not found https://www.dailydot.com/upstream/disney-plus-vs-hulu/
Error 404 file not found https://www.techradar.com/streaming/hulu-is-now-officially-part-of-disney-plus-heres-what-you-need-to-know
Error 404 file not found https://www.makeuseof.com/is-hulu-on-disney-worth-the-money/
Error 404 file not found https://deadline.com/2024/03/hulu-disney-plus-streaming-bundle-launch-1235869143/
Error 404 file not found https://www.toms